In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from otto_utils import *
from otto_features import *
from otto_candidates_covisit import *
from otto_lgbm_utils import *
from otto_implicit import *
from otto_reranker import *
from otto_jobs_candidates import *
from otto_jobs_datasets import *

In [3]:
df_test = pl.read_parquet(TEST_PROCESSED, use_pyarrow=True)
test_users = df_test["session"].unique().sort().to_list()

# Submissions

## carts + orders

In [6]:
action_type = "orders"
mode = "subm"
version = "+".join(["covisit_all", "tdidf_new_top_100", "tdidf_old_top_100"])
feature_groups = ["cand_user_item", "w2v_item2item", "covisit_item2item", "other"]

In [16]:
def lgb_cv_folds_predictions(
    df,
    model_file_tmplt,
    action_type,
    save_path: Path = Path("reranker_finale"),
):
    feature_cols = df.drop(["session", "aid"]).columns
    df_valid_preds = df.select(["session", "aid"])
    X_test = df[feature_cols].to_numpy()

    for fold in tqdm(range(5)):
#         model_file = model_file_tmplt.format(action_type=action_type, fold=fold)
        model_file = model_file_tmplt.format(action_type=action_type, fold=fold)
        gbm_ranking = lgb.Booster(model_file=(save_path / model_file).as_posix())
        scores = gbm_ranking.predict(X_test)
        df_valid_preds_fold = (
            df.select(["session", "aid"])
            .with_columns([pl.Series(scores).alias(f"scores_fold{fold}")])
        )
        df_valid_preds = (
            df_valid_preds.join(df_valid_preds_fold, on=["session", "aid"], how="left")
        )
    
    scores = (
        df_valid_preds
        .select([
            pl.col("scores_fold0"),
            pl.col("scores_fold1"),
            pl.col("scores_fold2"), 
            pl.col("scores_fold3"), 
            pl.col("scores_fold4"),
        ]).mean(axis=1)
    )
    
    df_valid_preds = (
        df_valid_preds.with_column(scores.alias(f"{action_type}_score"))
        .select(["session", "aid", f"{action_type}_score"])
    )
    return df_valid_preds

In [10]:
action_type = "orders"
model_version = "v2"
orders_model_file_tmplt = f"__model__{action_type}_{version}_reranker_fold{{fold}}_{model_version}.lgb"
print("orders tmplt: ", orders_model_file_tmplt)

action_type = "carts"
model_version = "v0"
carts_model_file_tmplt = f"__model__{action_type}_{version}_reranker_fold{{fold}}_{model_version}.lgb"
print("carts tmplt: ", carts_model_file_tmplt)

orders tmplt:  __model__orders_covisit_all+tdidf_new_top_100+tdidf_old_top_100_reranker_fold{fold}_v2.lgb
orders tmplt:  __model__carts_covisit_all+tdidf_new_top_100+tdidf_old_top_100_reranker_fold{fold}_v0.lgb


In [18]:
b_sz = 100000
# df_candidates_scores = df_candidates_item_item_features.select(["session", "aid"])

df_candidates_clicks_scores = []
df_candidates_carts_scores = []
df_candidates_orders_scores = []

action_type = "orders" # common dataset for carts and orders

for test_session_start in tqdm(range(0, len(test_users), b_sz)):
    test_sessions = test_users[test_session_start : test_session_start + b_sz]
    
    df_stage_2_dataset_batch = gather_dataset_features(
        mode = mode,
        action_type = action_type,
        version = version,
        feature_groups = feature_groups,
        batch_users=test_sessions,
    )

#     print("predict clicks -> save scores to df_candidates_scores")
#     df_clicks_scores = lgb_cv_folds_predictions(df_stage_2_batch, clicks_model_file_tmplt, "clicks")
#     df_candidates_clicks_scores.append(df_clicks_scores)
    
    print("predict carts -> save scores to df_candidates_scores")
    df_carts_scores = lgb_cv_folds_predictions(df_stage_2_dataset_batch, carts_model_file_tmplt, "carts")
    df_candidates_carts_scores.append(df_carts_scores)
    
    print("predict orders -> save scores to df_candidates_scores")
    df_orders_scores = lgb_cv_folds_predictions(df_stage_2_dataset_batch, orders_model_file_tmplt, "orders")
    df_candidates_orders_scores.append(df_orders_scores)


  0%|          | 0/17 [00:00<?, ?it/s]

mode:  subm
action type:  orders
version:  covisit_all+tdidf_new_top_100+tdidf_old_top_100
Reading generated candidate file...:  reranker_finale/__stage2__subm__orders__candidates_covisit_all+tdidf_new_top_100+tdidf_old_top_100.parquet
predict carts -> save scores to df_candidates_scores



100%|██████████| 5/5 [01:34<00:00, 18.94s/it]


predict orders -> save scores to df_candidates_scores



  6%|▌         | 1/17 [05:30<1:28:07, 330.48s/it]

...


 94%|█████████▍| 16/17 [1:27:39<05:28, 328.16s/it]

mode:  subm
action type:  orders
version:  covisit_all+tdidf_new_top_100+tdidf_old_top_100
Reading generated candidate file...:  reranker_finale/__stage2__subm__orders__candidates_covisit_all+tdidf_new_top_100+tdidf_old_top_100.parquet
predict carts -> save scores to df_candidates_scores



100%|██████████| 5/5 [01:06<00:00, 13.24s/it]


predict orders -> save scores to df_candidates_scores



100%|██████████| 17/17 [1:31:46<00:00, 323.92s/it]


In [19]:
df_candidates_carts_scores = pl.concat(df_candidates_carts_scores)
df_candidates_carts_scores.write_parquet(f"reranker_finale/__submission__carts_{version}_v0.parquet")

In [20]:
df_candidates_orders_scores = pl.concat(df_candidates_orders_scores)
df_candidates_orders_scores.write_parquet(f"reranker_finale/__submission__orders_{version}_v2.parquet")

## clicks

In [4]:
action_type = "clicks"
mode = "subm"
version = "+".join(["tdidf_new_top_100", "tdidf_old_top_100"])
feature_groups = ["cand_user_item", "w2v_item2item", "covisit_item2item", "other"]

In [5]:
def lgb_cv_folds_predictions(
    df,
    model_file_tmplt,
    action_type,
    save_path: Path = Path("reranker_finale"),
):
    feature_cols = df.drop(["session", "aid"]).columns
    df_valid_preds = df.select(["session", "aid"])
    X_test = df[feature_cols].to_numpy()

    for fold in tqdm(range(5)):
#         model_file = model_file_tmplt.format(action_type=action_type, fold=fold)
        model_file = model_file_tmplt.format(action_type=action_type, fold=fold)
        gbm_ranking = lgb.Booster(model_file=(save_path / model_file).as_posix())
        scores = gbm_ranking.predict(X_test)
        df_valid_preds_fold = (
            df.select(["session", "aid"])
            .with_columns([pl.Series(scores).alias(f"scores_fold{fold}")])
        )
        df_valid_preds = (
            df_valid_preds.join(df_valid_preds_fold, on=["session", "aid"], how="left")
        )
    
    scores = (
        df_valid_preds
        .select([
            pl.col("scores_fold0"),
            pl.col("scores_fold1"),
            pl.col("scores_fold2"), 
            pl.col("scores_fold3"), 
            pl.col("scores_fold4"),
        ]).mean(axis=1)
    )
    
    df_valid_preds = (
        df_valid_preds.with_column(scores.alias(f"{action_type}_score"))
        .select(["session", "aid", f"{action_type}_score"])
    )
    return df_valid_preds

In [6]:
action_type = "clicks"
model_version = "v0"
clicks_model_file_tmplt = f"__model__{action_type}_{version}_reranker_fold{{fold}}_{model_version}.lgb"
print("clicks tmplt: ", clicks_model_file_tmplt)

clicks tmplt:  __model__clicks_tdidf_new_top_100+tdidf_old_top_100_reranker_fold{fold}_v0.lgb


In [8]:
b_sz = 100000

df_candidates_clicks_scores = []
df_candidates_carts_scores = []
df_candidates_orders_scores = []

action_type = "clicks"

for test_session_start in tqdm(range(0, len(test_users), b_sz)):
    test_sessions = test_users[test_session_start : test_session_start + b_sz]
    
    df_stage_2_dataset_batch = gather_dataset_features(
        mode = mode,
        action_type = action_type,
        version = version,
        feature_groups = feature_groups,
        batch_users=test_sessions,
    )

    print("predict clicks -> save scores to df_candidates_scores")
    df_clicks_scores = lgb_cv_folds_predictions(df_stage_2_dataset_batch, clicks_model_file_tmplt, "clicks")
    df_candidates_clicks_scores.append(df_clicks_scores)


  0%|          | 0/17 [00:00<?, ?it/s]

mode:  subm
action type:  clicks
version:  tdidf_new_top_100+tdidf_old_top_100
Reading generated candidate file...:  reranker_finale/__stage2__subm__clicks__candidates_tdidf_new_top_100+tdidf_old_top_100.parquet
predict clicks -> save scores to df_candidates_scores



  6%|▌         | 1/17 [03:54<1:02:35, 234.72s/it]

mode:  subm
action type:  clicks
version:  tdidf_new_top_100+tdidf_old_top_100
Reading generated candidate file...:  reranker_finale/__stage2__subm__clicks__candidates_tdidf_new_top_100+tdidf_old_top_100.parquet
predict clicks -> save scores to df_candidates_scores



 12%|█▏        | 2/17 [06:35<47:45, 191.01s/it]  

...


 88%|████████▊ | 15/17 [41:31<05:23, 161.79s/it]

mode:  subm
action type:  clicks
version:  tdidf_new_top_100+tdidf_old_top_100
Reading generated candidate file...:  reranker_finale/__stage2__subm__clicks__candidates_tdidf_new_top_100+tdidf_old_top_100.parquet
predict clicks -> save scores to df_candidates_scores



 94%|█████████▍| 16/17 [44:12<02:41, 161.30s/it]

mode:  subm
action type:  clicks
version:  tdidf_new_top_100+tdidf_old_top_100
Reading generated candidate file...:  reranker_finale/__stage2__subm__clicks__candidates_tdidf_new_top_100+tdidf_old_top_100.parquet
predict clicks -> save scores to df_candidates_scores



100%|██████████| 17/17 [46:19<00:00, 163.48s/it]


In [9]:
df_candidates_clicks_scores = pl.concat(df_candidates_clicks_scores)
df_candidates_clicks_scores.write_parquet(f"reranker_finale/__submission__clicks_{version}_v0.parquet")

# Submission

## click

In [21]:
# df_candidates_clicks_scores = pl.read_parquet("__subm__clicks_scores_candidates_reranking_v3_1.parquet")
version = "+".join(["tdidf_new_top_100", "tdidf_old_top_100"])
df_candidates_clicks_scores = pl.read_parquet(f"reranker_finale/__submission__clicks_{version}_v0.parquet")

In [10]:
df_valid_preds_sorted = (
    df_candidates_clicks_scores
    .sort("clicks_score", reverse=True)
    .groupby("session")
    .agg([
        pl.list("aid"),
        pl.list("clicks_score"),
    ])
)

In [11]:
# stage 2 valid carts reranked - recall@20 optimized
submission_dict = {
    "session_type": [],
    "labels": [],
}

types = ["clicks"]
topk = 20

for row in tqdm(df_valid_preds_sorted.rows()):
    session_id = row[0]
    rec_items = row[1][:topk]
    
    session_types = [f"{session_id}_{t}" for t in types]
    labels = " ".join(str(aid) for aid in rec_items)
    labels_list = [labels]
    
    submission_dict["session_type"].extend(session_types)
    submission_dict["labels"].extend(labels_list)

100%|██████████| 1671803/1671803 [00:06<00:00, 258656.70it/s]


In [12]:
df_submission_clicks_reranked = pl.DataFrame(submission_dict)

In [13]:
df_submission_clicks_reranked.head()

session_type,labels
str,str
"""12990016_click...","""1678423 453451..."
"""13136960_click...","""824944 1460571..."
"""12977024_click...","""1650609 167637..."
"""14140032_click...","""1679462 184957..."
"""13433728_click...","""1180465 439222..."


## carts

In [14]:
version = "+".join(["covisit_all", "tdidf_new_top_100", "tdidf_old_top_100"])
df_candidates_carts_scores = pl.read_parquet(f"reranker_finale/__submission__carts_{version}_v0.parquet")

In [15]:
df_valid_preds_sorted = (
    df_candidates_carts_scores
    .sort("carts_score", reverse=True)
    .groupby("session")
    .agg([
        pl.list("aid"),
        pl.list("carts_score"),
    ])
)

In [16]:
df_valid_preds_sorted

session,aid,carts_score
i64,list[i64],list[f64]
14350144,"[337471, 1466369, ... 984617]","[1.814851, -0.477041, ... -10.662985]"
13535488,"[1692706, 1142507, ... 1243358]","[3.183787, 0.517683, ... -11.981055]"
13165696,"[633760, 309703, ... 1679400]","[2.96021, 1.857757, ... -38.552639]"
14139072,"[378765, 1840567, ... 477922]","[2.778367, -0.397153, ... -11.058259]"
14086208,"[1768879, 671101, ... 361995]","[1.535539, 0.840866, ... -8.710822]"
13757440,"[859334, 1117208, ... 121534]","[3.208259, 0.84079, ... -10.911554]"
13039040,"[749432, 268669, ... 1688832]","[2.950695, 1.282091, ... -9.715829]"
13259456,"[215649, 1322823, ... 125524]","[4.219167, 3.040282, ... -7.666617]"
13976192,"[62659, 1039611, ... 1823626]","[3.15935, 1.95204, ... -10.939]"


In [17]:
# stage 2 valid carts reranked - recall@20 optimized
submission_dict = {
    "session_type": [],
    "labels": [],
}

types = ["carts"]
topk = 20

for row in tqdm(df_valid_preds_sorted.rows()):
    session_id = row[0]
    rec_items = row[1][:topk]
    
    session_types = [f"{session_id}_{t}" for t in types]
    labels = " ".join(str(aid) for aid in rec_items)
    labels_list = [labels]
    
    submission_dict["session_type"].extend(session_types)
    submission_dict["labels"].extend(labels_list)

100%|██████████| 1671803/1671803 [00:06<00:00, 260266.38it/s]


In [18]:
df_submission_carts_reranked = pl.DataFrame(submission_dict)

In [19]:
df_submission_carts_reranked.head()

session_type,labels
str,str
"""14350144_carts...","""337471 1466369..."
"""13535488_carts...","""1692706 114250..."
"""13165696_carts...","""633760 309703 ..."
"""14139072_carts...","""378765 1840567..."
"""14086208_carts...","""1768879 671101..."


## orders

In [20]:
version = "+".join(["covisit_all", "tdidf_new_top_100", "tdidf_old_top_100"])
df_candidates_orders_scores = pl.read_parquet(f"reranker_finale/__submission__orders_{version}_v2.parquet")

In [21]:
df_valid_preds_sorted = (
    df_candidates_orders_scores
    .sort("orders_score", reverse=True)
    .groupby("session")
    .agg([
        pl.list("aid"),
        pl.list("orders_score"),
    ])
)

In [22]:
df_valid_preds_sorted.head()

session,aid,orders_score
i64,list[i64],list[f64]
14424768,"[885917, 350578, ... 152547]","[3.154448, 1.613243, ... -10.283263]"
13432832,"[469367, 180672, ... 152547]","[3.047045, -0.751042, ... -10.226279]"
13355392,"[1286372, 554623, ... 152547]","[4.787866, 0.194021, ... -9.799678]"
14112832,"[335640, 882058, ... 152547]","[2.586109, 1.370388, ... -9.899193]"
13729472,"[394862, 1127676, ... 152547]","[3.593447, 1.387175, ... -10.165423]"


In [23]:
# stage 2 valid carts reranked - recall@20 optimized
submission_dict = {
    "session_type": [],
    "labels": [],
}

types = ["orders"]
topk = 20

for row in tqdm(df_valid_preds_sorted.rows()):
    session_id = row[0]
    rec_items = row[1][:topk]
    
    session_types = [f"{session_id}_{t}" for t in types]
    labels = " ".join(str(aid) for aid in rec_items)
    labels_list = [labels]
    
    submission_dict["session_type"].extend(session_types)
    submission_dict["labels"].extend(labels_list)

100%|██████████| 1671803/1671803 [00:06<00:00, 260444.13it/s]


In [24]:
df_submission_orders_reranked = pl.DataFrame(submission_dict)

In [25]:
df_submission_orders_reranked.head()

session_type,labels
str,str
"""14424768_order...","""885917 350578 ..."
"""13432832_order...","""469367 180672 ..."
"""13355392_order...","""1286372 554623..."
"""14112832_order...","""335640 882058 ..."
"""13729472_order...","""394862 1127676..."


## Combine

### clicks, carts, orders

In [26]:
df_submission_reranked_all = (
    pl.from_pandas(
        pd.read_csv("covisit_top200_reranker_rank+42feat_submission.csv.gz", compression="gzip")
    )
    .join(
        pl.concat([
            df_submission_clicks_reranked,
            df_submission_carts_reranked,
            df_submission_orders_reranked
        ]), 
        on="session_type", how="left"
    )
    .with_columns([
        (
            pl.when(pl.col("labels_right").is_null())
            .then(pl.col("labels"))
            .otherwise(pl.col("labels_right"))
        ).alias("labels_final")
    ])
    .select(["session_type", "labels_final"])
    .rename({"labels_final": "labels"})
)


In [27]:
df_submission_reranked_all.head()

session_type,labels
str,str
"""12899779_click...","""59625 737445 1..."
"""12899779_carts...","""59625 731692 1..."
"""12899779_order...","""59625 731692 6..."
"""12899780_click...","""1142000 736515..."
"""12899780_carts...","""1142000 582732..."


In [28]:
# LB - 0.595 (BEST)
(
    df_submission_reranked_all.to_pandas()
    .to_csv("reranker_finale/__b__submission_finale.csv.gz",
            compression="gzip", index=False)
)

In [38]:
# LB - 0.595
(
    df_submission_reranked_all.to_pandas()
    .to_csv("reranker_finale/__a__submission_finale.csv.gz",
            compression="gzip", index=False)
)